In [38]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv("Indian_Kids_Screen_Time_Cleaned.csv")

In [40]:
exceeded = df[df['Exceeded_Recommended_Limit'] == True]
min_exceeded = exceeded['Avg_Daily_Screen_Time_hr'].min()
print("Minimum Screen Time (Exceeded Limit):", min_exceeded)

Minimum Screen Time (Exceeded Limit): 2.01


In [41]:
not_exceeded = df[df['Exceeded_Recommended_Limit'] == False]
max_not_exceeded = not_exceeded['Avg_Daily_Screen_Time_hr'].max()
print("Minimum Screen Time (Exceeded Limit):", max_not_exceeded)

Minimum Screen Time (Exceeded Limit): 3.0


NO OVERLAP

In [42]:
filtered_data = df[(df['Avg_Daily_Screen_Time_hr'] >= 2.01) & (df['Avg_Daily_Screen_Time_hr'] <= 3)]
filtered_data[['Avg_Daily_Screen_Time_hr', 'Exceeded_Recommended_Limit']]

,Avg_Daily_Screen_Time_hr,Exceeded_Recommended_Limit
6,2.97,False
7,2.74,True
32,2.16,True
44,2.75,False
47,2.34,False
...,...,...
9625,2.50,False
9643,2.33,True
9647,2.26,True
9651,2.69,False


In [43]:
avg_screen_time_fd = filtered_data['Avg_Daily_Screen_Time_hr'].mean()
print("Average Screen Time (2.01–3 hr range):", avg_screen_time_fd)

Average Screen Time (2.01–3 hr range): 2.5694021739130433


In [44]:
df[(df['Avg_Daily_Screen_Time_hr']> avg_screen_time_fd) & (df['Avg_Daily_Screen_Time_hr']<max_not_exceeded)]

,Age,Gender,Avg_Daily_Screen_Time_hr,Primary_Device,Exceeded_Recommended_Limit,Educational_to_Recreational_Ratio,Health_Impacts,Urban_or_Rural
6,17,Male,2.97,TV,False,0.48,"[""['No Impact']""]",Rural
7,10,Male,2.74,TV,True,0.54,"[""['No Impact']""]",Urban
44,11,Male,2.75,Smartphone,False,0.35,"[""['No Impact']""]",Rural
48,16,Male,2.85,Smartphone,False,0.30,"[""['No Impact']""]",Rural
52,11,Male,2.83,Smartphone,False,0.40,"[""['No Impact']""]",Urban
...,...,...,...,...,...,...,...,...
9535,8,Female,2.83,Tablet,True,0.43,"[""['No Impact']""]",Urban
9540,14,Male,2.63,Laptop,False,0.33,"[""['No Impact']""]",Urban
9606,18,Male,2.84,Laptop,False,0.36,"[""['No Impact']""]",Urban
9651,12,Female,2.69,Smartphone,False,0.48,"[""['No Impact']""]",Urban


In [45]:
df['Age'].max()

18

# ----------------------------------------------------------------
# SCREEN TIME RECOMMENDATION SYSTEM 
# ----------------------------------------------------------------


In [9]:
import pandas as pd
import numpy as np


In [2]:
# STEP 1: Load Dataset
# --------------------------------------------
df = pd.read_csv("Indian_Kids_Screen_Time_Cleaned.csv")

# Quick look
print("Dataset loaded successfully with shape:", df.shape)



Dataset loaded successfully with shape: (9668, 8)


In [3]:
# STEP 2: Health Impact Insights
# --------------------------------------------
# Split multiple health impacts into separate rows
df['Health_Impacts'] = df['Health_Impacts'].astype(str).str.replace(r"[\[\]']", '', regex=True)
df['Health_Impacts'] = df['Health_Impacts'].str.split(',')

# Explode to separate rows for each health impact
df_exploded = df.explode('Health_Impacts')
df_exploded['Health_Impacts'] = df_exploded['Health_Impacts'].str.strip()

# Calculate average and minimum screen time per health impact
avg_screen_time_per_impact = (
    df_exploded.groupby('Health_Impacts')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .sort_values(by='Avg_Daily_Screen_Time_hr', ascending=False)
)

min_screen_time_per_impact = (
    df_exploded.groupby('Health_Impacts')['Avg_Daily_Screen_Time_hr']
    .min()
    .reset_index()
)

print("\n📊 Average Screen Time per Health Impact:\n", avg_screen_time_per_impact)
print("\n🔹 Minimum Screen Time per Health Impact:\n", min_screen_time_per_impact)

# Clean up spaces and quotes in Health_Impacts
min_screen_time_per_impact['Health_Impacts'] = (
    min_screen_time_per_impact['Health_Impacts']
    .str.replace('"', '')    # remove quotes
    .str.strip()             # remove extra spaces
)

# Find the minimum screen time (excluding 'No Impact')
min_limit = min_screen_time_per_impact[
    min_screen_time_per_impact['Health_Impacts'] != 'No Impact'
]['Avg_Daily_Screen_Time_hr'].min()

print(f"\n⚖️ Recommended Health-based Screen Time Limit (below): {min_limit:.2f} hrs")


📊 Average Screen Time per Health Impact:
      Health_Impacts  Avg_Daily_Screen_Time_hr
7      "Poor Sleep"                  4.931915
0       "  Anxiety"                  4.930052
2  "  Obesity Risk"                  4.928591
6    "Obesity Risk"                  4.911151
4      "Eye Strain"                  4.872391
1    "  Eye Strain"                  4.870691
3         "Anxiety"                  4.864714
5       "No Impact"                  3.245264

🔹 Minimum Screen Time per Health Impact:
      Health_Impacts  Avg_Daily_Screen_Time_hr
0       "  Anxiety"                      3.01
1    "  Eye Strain"                      3.01
2  "  Obesity Risk"                      3.02
3         "Anxiety"                      3.01
4      "Eye Strain"                      3.02
5       "No Impact"                      0.00
6    "Obesity Risk"                      3.01
7      "Poor Sleep"                      3.01

⚖️ Recommended Health-based Screen Time Limit (below): 3.01 hrs


In [4]:
# STEP 3: Precompute Age and Device Insights
# --------------------------------------------
non_exceeded = df[df['Exceeded_Recommended_Limit'] == False]

# Average (not max) screen time per age among non-exceeded users
avg_screen_time_age = (
    non_exceeded.groupby('Age')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .rename(columns={'Avg_Daily_Screen_Time_hr': 'Avg_Screen_Time_By_Age'})
)

# Average screen time per device among non-exceeded users
avg_screen_time_device = (
    non_exceeded.groupby('Primary_Device')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .rename(columns={'Avg_Daily_Screen_Time_hr': 'Avg_Screen_Time_By_Device'})
)

# Convert to dictionary for lookup
age_to_limit = dict(zip(avg_screen_time_age['Age'], avg_screen_time_age['Avg_Screen_Time_By_Age']))
device_to_limit = dict(zip(avg_screen_time_device['Primary_Device'], avg_screen_time_device['Avg_Screen_Time_By_Device']))

# Preview
print("\n📊 Age-based Average Screen Times:\n", avg_screen_time_age)
print("\n💻 Device-based Average Screen Times:\n", avg_screen_time_device)


📊 Age-based Average Screen Times:
     Age  Avg_Screen_Time_By_Age
0     8                0.816129
1     9                0.792105
2    10                0.765937
3    11                2.374205
4    12                2.384353
5    13                2.430211
6    14                2.378471
7    15                2.436316
8    16                2.401026
9    17                2.325446
10   18                2.483875

💻 Device-based Average Screen Times:
   Primary_Device  Avg_Screen_Time_By_Device
0         Laptop                   2.320000
1     Smartphone                   1.723758
2             TV                   1.332785
3         Tablet                   1.347357


In [5]:
# STEP 4: Define Helper Functions
# --------------------------------------------

def get_age_based_limit(age):
    """Recommended limit (hrs) based on age average, capped below 3.01."""
    base_limit = age_to_limit.get(age, df['Avg_Daily_Screen_Time_hr'].mean())
    return round(min(base_limit, 3.00), 2)

def get_device_based_limit(device):
    """Recommended limit (hrs) based on device average, capped below 3.01."""
    base_limit = device_to_limit.get(device, df['Avg_Daily_Screen_Time_hr'].mean())
    return round(min(base_limit, 3.00), 2)

def get_combined_recommended_limit(age, device):
    """Weighted combined limit of both (50%-50%), capped below 3.01."""
    combined_limit = (0.5 * get_age_based_limit(age)) + (0.5 * get_device_based_limit(device))
    return round(min(combined_limit, 3.00), 2)


def exceeded_logic(age, device, screen_time):
    """Determine exceed status based on each limit."""
    age_limit = get_age_based_limit(age)
    device_limit = get_device_based_limit(device)
    combined_limit = get_combined_recommended_limit(age, device)

    return (
        screen_time > age_limit,
        screen_time > device_limit,
        screen_time > combined_limit
    )

In [6]:
# STEP 5: Main Recommendation System
# --------------------------------------------
def recommendation_system(age, gender, device, screen_time):
    """Returns detailed personalized report."""

    # Percentile: how many users have lower screen time
    users_behind = round((df['Avg_Daily_Screen_Time_hr'] < screen_time).mean() * 100, 2)

    # Gender-wise and device-wise averages
    gender_avg = df[df['Gender'] == gender]['Avg_Daily_Screen_Time_hr'].mean()
    device_avg = df[df['Primary_Device'] == device]['Avg_Daily_Screen_Time_hr'].mean()

    # Limits
    age_limit = get_age_based_limit(age)
    device_limit = get_device_based_limit(device)
    combined_limit = get_combined_recommended_limit(age, device)

    # Exceeded checks
    exceeded_age, exceeded_device, exceeded_combined = exceeded_logic(age, device, screen_time)

    return {
        "Age": age,
        "Gender": gender,
        "Device": device,
        "Your Screen Time (hrs)": screen_time,
        "Exceeded Age Limit": exceeded_age,
        "Exceeded Device Limit": exceeded_device,
        "Exceeded Combined Limit": exceeded_combined,
        "Users Behind (%)": users_behind,
        "Age-based Recommended Limit (hrs)": age_limit,
        "Device-based Recommended Limit (hrs)": device_limit,
        "Combined Recommended Limit (hrs)": combined_limit,
        "Gender-wise Avg Screen Time (hrs)": round(gender_avg, 2),
        "Device-wise Avg Screen Time (hrs)": round(device_avg, 2)
    }


In [7]:
# STEP 6: User Input
# --------------------------------------------
print("\nWelcome to the Enhanced Screen Time Recommendation System ⚡")
user_age = int(input("Enter your age: "))
user_gender = input("Enter your gender (Male/Female): ").capitalize()
user_device = input("Enter your primary device (Smartphone/Laptop/TV etc.): ").capitalize()
user_screen_time = float(input("Enter your average daily screen time (in hours): "))



Welcome to the Enhanced Screen Time Recommendation System ⚡
Enter your age: 12
Enter your gender (Male/Female): Male
Enter your primary device (Smartphone/Laptop/TV etc.): TV
Enter your average daily screen time (in hours): 2.09


In [8]:
# STEP 7: Generate Report
# --------------------------------------------
result = recommendation_system(user_age, user_gender, user_device, user_screen_time)

print("\n--- RECOMMENDATION REPORT ---")
for k, v in result.items():
    print(f"{k}: {v}")

# Summary Message
diff = result['Your Screen Time (hrs)'] - result['Combined Recommended Limit (hrs)']
if diff > 0:
    print(f"\n⚠️ You are spending {diff:.2f} hrs more than the recommended safe limit (<3.01 hrs). Try reducing it.")
else:
    print(f"\n✅ Great! You are {abs(diff):.2f} hrs below the recommended safe limit.")



--- RECOMMENDATION REPORT ---
Age: 12
Gender: Male
Device: Tv
Your Screen Time (hrs): 2.09
Exceeded Age Limit: False
Exceeded Device Limit: False
Exceeded Combined Limit: False
Users Behind (%): 8.8
Age-based Recommended Limit (hrs): 2.38
Device-based Recommended Limit (hrs): 3.0
Combined Recommended Limit (hrs): 2.69
Gender-wise Avg Screen Time (hrs): 4.39
Device-wise Avg Screen Time (hrs): nan

✅ Great! You are 0.60 hrs below the recommended safe limit.
